In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
# Source: 
# https://www.learnopencv.com/image-classification-using-transfer-learning-in-pytorch/

# Applying Transforms to the Data
image_transforms = {
    'train': transforms.Compose([
#         transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
#         transforms.RandomRotation(degrees=15),
#         transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}


# Set train and valid directory paths
train_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/train'
valid_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/valid'
test_directory = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/test'
 
# Batch size
bs = 64
 
# Number of classes
num_classes = 2
 
# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid']),
    'test': datasets.ImageFolder(root=test_directory, transform=image_transforms['test'])
}
 
# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])
 
# Create iterators for the Data loaded using DataLoader module
train_data = torch.utils.data.DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data = torch.utils.data.DataLoader(data['valid'], batch_size=bs, shuffle=True)
test_data = torch.utils.data.DataLoader(data['test'], batch_size=bs, shuffle=True)
 
# Print the train, validation and test set data sizes
train_data_size, valid_data_size, test_data_size

(29828, 4000, 4000)

In [3]:
# Load pretrained AlexNet Model
alexnet = torchvision.models.alexnet(pretrained=True)

In [4]:
# Source: 
# https://analyticsindiamag.com/implementing-alexnet-using-pytorch-as-a-transfer-learning-model-in-multi-class-classification/

# Updating the second classifier
alexnet.classifier[4] = nn.Linear(4096,1024)

#Updating the third and the last classifier that is the output layer of the network. Make sure to have 10 output nodes if we are going to get 10 class labels through our model.
# AdVo: i will have a binary classification 
alexnet.classifier[6] = nn.Linear(1024,num_classes)

In [5]:
#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# for epoch in range(10):  # loop over the dataset multiple times
#     running_loss = 0.0
#     for i, data in enumerate(train_data, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data[0].to(device), data[1].to(device)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         output = alexnet(inputs)
#         loss = criterion(output, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

# print('Finished Training of AlexNet')

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
for epoch in range(10):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, 10))
     
    # Set to training mode
    alexnet.train()
     
    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0
     
    valid_loss = 0.0
    valid_acc = 0.0
 
    for i, (inputs, labels) in enumerate(train_data):
 
        inputs = inputs.to(device)
        labels = labels.to(device)
         
        # Clean existing gradients
        optimizer.zero_grad()
         
        # Forward pass - compute outputs on input data using the model
        outputs = alexnet(inputs)
         
        # Compute loss
        loss = criterion(outputs, labels)
         
        # Backpropagate the gradients
        loss.backward()
         
        # Update the parameters
        optimizer.step()
         
        # Compute the total loss for the batch and add it to train_loss
        train_loss += loss.item() * inputs.size(0)
         
        # Compute the accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
         
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
         
        # Compute total accuracy in the whole batch and add to train_acc
        train_acc += acc.item() * inputs.size(0)
         
        print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

Epoch: 1/10
Batch number: 000, Training: Loss: 0.7151, Accuracy: 0.5156
Batch number: 001, Training: Loss: 0.7281, Accuracy: 0.4531
Batch number: 002, Training: Loss: 0.6855, Accuracy: 0.5781
Batch number: 003, Training: Loss: 0.6976, Accuracy: 0.4375
Batch number: 004, Training: Loss: 0.7167, Accuracy: 0.4375
Batch number: 005, Training: Loss: 0.6840, Accuracy: 0.5156
Batch number: 006, Training: Loss: 0.6301, Accuracy: 0.6719
Batch number: 007, Training: Loss: 0.6464, Accuracy: 0.6250
Batch number: 008, Training: Loss: 0.6579, Accuracy: 0.6406
Batch number: 009, Training: Loss: 0.6719, Accuracy: 0.5156
Batch number: 010, Training: Loss: 0.6026, Accuracy: 0.7031
Batch number: 011, Training: Loss: 0.6250, Accuracy: 0.6250
Batch number: 012, Training: Loss: 0.5556, Accuracy: 0.7031
Batch number: 013, Training: Loss: 0.5733, Accuracy: 0.7031
Batch number: 014, Training: Loss: 0.5338, Accuracy: 0.7656
Batch number: 015, Training: Loss: 0.4824, Accuracy: 0.8281
Batch number: 016, Training:

Batch number: 137, Training: Loss: 0.0323, Accuracy: 1.0000
Batch number: 138, Training: Loss: 0.2219, Accuracy: 0.9375
Batch number: 139, Training: Loss: 0.0899, Accuracy: 0.9531
Batch number: 140, Training: Loss: 0.0341, Accuracy: 1.0000
Batch number: 141, Training: Loss: 0.1594, Accuracy: 0.9531
Batch number: 142, Training: Loss: 0.2101, Accuracy: 0.9219
Batch number: 143, Training: Loss: 0.1906, Accuracy: 0.9219
Batch number: 144, Training: Loss: 0.0423, Accuracy: 0.9844
Batch number: 145, Training: Loss: 0.1240, Accuracy: 0.9688
Batch number: 146, Training: Loss: 0.1756, Accuracy: 0.9375
Batch number: 147, Training: Loss: 0.2260, Accuracy: 0.9062
Batch number: 148, Training: Loss: 0.1025, Accuracy: 0.9531
Batch number: 149, Training: Loss: 0.0570, Accuracy: 0.9688
Batch number: 150, Training: Loss: 0.3724, Accuracy: 0.8750
Batch number: 151, Training: Loss: 0.1037, Accuracy: 0.9688
Batch number: 152, Training: Loss: 0.1117, Accuracy: 0.9531
Batch number: 153, Training: Loss: 0.131

Batch number: 274, Training: Loss: 0.0890, Accuracy: 0.9688
Batch number: 275, Training: Loss: 0.0382, Accuracy: 0.9844
Batch number: 276, Training: Loss: 0.0730, Accuracy: 0.9844
Batch number: 277, Training: Loss: 0.0960, Accuracy: 0.9688
Batch number: 278, Training: Loss: 0.0323, Accuracy: 0.9844
Batch number: 279, Training: Loss: 0.0595, Accuracy: 0.9844
Batch number: 280, Training: Loss: 0.0956, Accuracy: 0.9531
Batch number: 281, Training: Loss: 0.0801, Accuracy: 0.9688
Batch number: 282, Training: Loss: 0.2519, Accuracy: 0.9531
Batch number: 283, Training: Loss: 0.1978, Accuracy: 0.9375
Batch number: 284, Training: Loss: 0.1346, Accuracy: 0.9531
Batch number: 285, Training: Loss: 0.0262, Accuracy: 1.0000
Batch number: 286, Training: Loss: 0.0429, Accuracy: 1.0000
Batch number: 287, Training: Loss: 0.0856, Accuracy: 0.9375
Batch number: 288, Training: Loss: 0.0352, Accuracy: 1.0000
Batch number: 289, Training: Loss: 0.0429, Accuracy: 1.0000
Batch number: 290, Training: Loss: 0.136

Batch number: 411, Training: Loss: 0.0194, Accuracy: 1.0000
Batch number: 412, Training: Loss: 0.0184, Accuracy: 1.0000
Batch number: 413, Training: Loss: 0.0832, Accuracy: 0.9688
Batch number: 414, Training: Loss: 0.0426, Accuracy: 0.9844
Batch number: 415, Training: Loss: 0.0448, Accuracy: 0.9844
Batch number: 416, Training: Loss: 0.0180, Accuracy: 1.0000
Batch number: 417, Training: Loss: 0.0740, Accuracy: 0.9688
Batch number: 418, Training: Loss: 0.0592, Accuracy: 0.9844
Batch number: 419, Training: Loss: 0.0744, Accuracy: 0.9688
Batch number: 420, Training: Loss: 0.1840, Accuracy: 0.9531
Batch number: 421, Training: Loss: 0.1074, Accuracy: 0.9688
Batch number: 422, Training: Loss: 0.0635, Accuracy: 0.9688
Batch number: 423, Training: Loss: 0.0259, Accuracy: 0.9844
Batch number: 424, Training: Loss: 0.0644, Accuracy: 0.9688
Batch number: 425, Training: Loss: 0.0161, Accuracy: 1.0000
Batch number: 426, Training: Loss: 0.0638, Accuracy: 0.9844
Batch number: 427, Training: Loss: 0.093

Batch number: 081, Training: Loss: 0.0180, Accuracy: 1.0000
Batch number: 082, Training: Loss: 0.0458, Accuracy: 0.9844
Batch number: 083, Training: Loss: 0.1058, Accuracy: 0.9844
Batch number: 084, Training: Loss: 0.0587, Accuracy: 0.9844
Batch number: 085, Training: Loss: 0.0058, Accuracy: 1.0000
Batch number: 086, Training: Loss: 0.0163, Accuracy: 1.0000
Batch number: 087, Training: Loss: 0.0356, Accuracy: 0.9844
Batch number: 088, Training: Loss: 0.0862, Accuracy: 0.9531
Batch number: 089, Training: Loss: 0.0813, Accuracy: 0.9688
Batch number: 090, Training: Loss: 0.0284, Accuracy: 1.0000
Batch number: 091, Training: Loss: 0.0182, Accuracy: 1.0000
Batch number: 092, Training: Loss: 0.1952, Accuracy: 0.9688
Batch number: 093, Training: Loss: 0.0572, Accuracy: 0.9688
Batch number: 094, Training: Loss: 0.0558, Accuracy: 0.9688
Batch number: 095, Training: Loss: 0.0495, Accuracy: 0.9688
Batch number: 096, Training: Loss: 0.0254, Accuracy: 0.9844
Batch number: 097, Training: Loss: 0.005

...

Batch number: 374, Training: Loss: 0.0023, Accuracy: 1.0000
Batch number: 375, Training: Loss: 0.0013, Accuracy: 1.0000
Batch number: 376, Training: Loss: 0.0051, Accuracy: 1.0000
Batch number: 377, Training: Loss: 0.0029, Accuracy: 1.0000
Batch number: 378, Training: Loss: 0.0060, Accuracy: 1.0000
Batch number: 379, Training: Loss: 0.0632, Accuracy: 0.9844
Batch number: 380, Training: Loss: 0.0504, Accuracy: 0.9844
Batch number: 381, Training: Loss: 0.0008, Accuracy: 1.0000
Batch number: 382, Training: Loss: 0.0018, Accuracy: 1.0000
Batch number: 383, Training: Loss: 0.0292, Accuracy: 0.9844
Batch number: 384, Training: Loss: 0.0087, Accuracy: 1.0000
Batch number: 385, Training: Loss: 0.0103, Accuracy: 1.0000
Batch number: 386, Training: Loss: 0.0028, Accuracy: 1.0000
Batch number: 387, Training: Loss: 0.1509, Accuracy: 0.9844
Batch number: 388, Training: Loss: 0.0401, Accuracy: 0.9844
Batch number: 389, Training: Loss: 0.0017, Accuracy: 1.0000
Batch number: 390, Training: Loss: 0.038

Batch number: 044, Training: Loss: 0.0043, Accuracy: 1.0000
Batch number: 045, Training: Loss: 0.0017, Accuracy: 1.0000
Batch number: 046, Training: Loss: 0.0007, Accuracy: 1.0000
Batch number: 047, Training: Loss: 0.0030, Accuracy: 1.0000
Batch number: 048, Training: Loss: 0.0096, Accuracy: 1.0000
Batch number: 049, Training: Loss: 0.0022, Accuracy: 1.0000
Batch number: 050, Training: Loss: 0.0006, Accuracy: 1.0000
Batch number: 051, Training: Loss: 0.0036, Accuracy: 1.0000
Batch number: 052, Training: Loss: 0.0006, Accuracy: 1.0000
Batch number: 053, Training: Loss: 0.0028, Accuracy: 1.0000
Batch number: 054, Training: Loss: 0.0071, Accuracy: 1.0000
Batch number: 055, Training: Loss: 0.0004, Accuracy: 1.0000
Batch number: 056, Training: Loss: 0.0004, Accuracy: 1.0000
Batch number: 057, Training: Loss: 0.0056, Accuracy: 1.0000
Batch number: 058, Training: Loss: 0.0006, Accuracy: 1.0000
Batch number: 059, Training: Loss: 0.0016, Accuracy: 1.0000
Batch number: 060, Training: Loss: 0.016

Batch number: 181, Training: Loss: 0.0050, Accuracy: 1.0000
Batch number: 182, Training: Loss: 0.1263, Accuracy: 0.9844
Batch number: 183, Training: Loss: 0.0013, Accuracy: 1.0000
Batch number: 184, Training: Loss: 0.0017, Accuracy: 1.0000
Batch number: 185, Training: Loss: 0.0029, Accuracy: 1.0000
Batch number: 186, Training: Loss: 0.0093, Accuracy: 1.0000
Batch number: 187, Training: Loss: 0.0014, Accuracy: 1.0000
Batch number: 188, Training: Loss: 0.0052, Accuracy: 1.0000
Batch number: 189, Training: Loss: 0.0041, Accuracy: 1.0000
Batch number: 190, Training: Loss: 0.0288, Accuracy: 0.9844
Batch number: 191, Training: Loss: 0.0210, Accuracy: 0.9844
Batch number: 192, Training: Loss: 0.0021, Accuracy: 1.0000
Batch number: 193, Training: Loss: 0.0009, Accuracy: 1.0000
Batch number: 194, Training: Loss: 0.0195, Accuracy: 0.9844
Batch number: 195, Training: Loss: 0.0013, Accuracy: 1.0000
Batch number: 196, Training: Loss: 0.0038, Accuracy: 1.0000
Batch number: 197, Training: Loss: 0.090

Batch number: 318, Training: Loss: 0.0085, Accuracy: 1.0000
Batch number: 319, Training: Loss: 0.0003, Accuracy: 1.0000
Batch number: 320, Training: Loss: 0.0454, Accuracy: 0.9844
Batch number: 321, Training: Loss: 0.0186, Accuracy: 0.9844
Batch number: 322, Training: Loss: 0.0050, Accuracy: 1.0000
Batch number: 323, Training: Loss: 0.0634, Accuracy: 0.9844
Batch number: 324, Training: Loss: 0.0043, Accuracy: 1.0000
Batch number: 325, Training: Loss: 0.0048, Accuracy: 1.0000
Batch number: 326, Training: Loss: 0.0032, Accuracy: 1.0000
Batch number: 327, Training: Loss: 0.0072, Accuracy: 1.0000
Batch number: 328, Training: Loss: 0.0102, Accuracy: 1.0000
Batch number: 329, Training: Loss: 0.0030, Accuracy: 1.0000
Batch number: 330, Training: Loss: 0.0038, Accuracy: 1.0000
Batch number: 331, Training: Loss: 0.0059, Accuracy: 1.0000
Batch number: 332, Training: Loss: 0.0051, Accuracy: 1.0000
Batch number: 333, Training: Loss: 0.0119, Accuracy: 1.0000
Batch number: 334, Training: Loss: 0.004

Batch number: 455, Training: Loss: 0.0062, Accuracy: 1.0000
Batch number: 456, Training: Loss: 0.0793, Accuracy: 0.9844
Batch number: 457, Training: Loss: 0.0148, Accuracy: 1.0000
Batch number: 458, Training: Loss: 0.0037, Accuracy: 1.0000
Batch number: 459, Training: Loss: 0.0070, Accuracy: 1.0000
Batch number: 460, Training: Loss: 0.0096, Accuracy: 1.0000
Batch number: 461, Training: Loss: 0.0050, Accuracy: 1.0000
Batch number: 462, Training: Loss: 0.0563, Accuracy: 0.9844
Batch number: 463, Training: Loss: 0.0796, Accuracy: 0.9844
Batch number: 464, Training: Loss: 0.0053, Accuracy: 1.0000
Batch number: 465, Training: Loss: 0.0071, Accuracy: 1.0000
Batch number: 466, Training: Loss: 0.0203, Accuracy: 1.0000


In [8]:
history = []
# Validation - No gradient tracking needed
with torch.no_grad():
 
    # Set to evaluation mode
    alexnet.eval()
 
    # Validation loop
    for j, (inputs, labels) in enumerate(valid_data):
        inputs = inputs.to(device)
        labels = labels.to(device)
 
        # Forward pass - compute outputs on input data using the model
        outputs = alexnet(inputs)
 
        # Compute loss
        loss = criterion(outputs, labels)
 
        # Compute the total loss for the batch and add it to valid_loss
        valid_loss += loss.item() * inputs.size(0)
 
        # Calculate validation accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
 
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
 
        # Compute total accuracy in the whole batch and add to valid_acc
        valid_acc += acc.item() * inputs.size(0)
 
        print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
     
# Find average training loss and training accuracy
avg_train_loss = train_loss/train_data_size
avg_train_acc = train_acc/float(train_data_size)
 
# Find average training loss and training accuracy
avg_valid_loss = valid_loss/valid_data_size
avg_valid_acc = valid_acc/float(valid_data_size)
 
history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
         
epoch_end = time.time()
 
print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

Validation Batch number: 000, Validation: Loss: 0.0257, Accuracy: 1.0000
Validation Batch number: 001, Validation: Loss: 0.0465, Accuracy: 0.9844
Validation Batch number: 002, Validation: Loss: 0.0354, Accuracy: 1.0000
Validation Batch number: 003, Validation: Loss: 0.0421, Accuracy: 0.9688
Validation Batch number: 004, Validation: Loss: 0.1000, Accuracy: 0.9531
Validation Batch number: 005, Validation: Loss: 0.0394, Accuracy: 0.9844
Validation Batch number: 006, Validation: Loss: 0.0255, Accuracy: 0.9844
Validation Batch number: 007, Validation: Loss: 0.0297, Accuracy: 0.9844
Validation Batch number: 008, Validation: Loss: 0.0455, Accuracy: 0.9844
Validation Batch number: 009, Validation: Loss: 0.0538, Accuracy: 0.9844
Validation Batch number: 010, Validation: Loss: 0.0213, Accuracy: 1.0000
Validation Batch number: 011, Validation: Loss: 0.0077, Accuracy: 1.0000
Validation Batch number: 012, Validation: Loss: 0.0144, Accuracy: 1.0000
Validation Batch number: 013, Validation: Loss: 0.1